In [ ]:
# Import tdmclient Notebook environment:
import tdmclient.notebook
#from ipywidgets import interact, interactive, fixed, interact_manual
#import ipywidgets as widgets
import numpy as np
await tdmclient.notebook.start()

%%run_python

speed0 = 100       # nominal speed
obstThrL = 500      # low obstacle threshold to switch state 1->0
obstThrH = 900      # high obstacle threshold to switch state 0->1
obstSpeedGain = [10, 6, 0, -6, -10]  # /100 

state = 0          # 0=A* algorithm, 1=obstacle avoidance
obst = [0,0,0,0,0]       # measurements from left and right prox sensors



#timer_period[0] = 10   # 10ms sampling time

@onevent 
def obstacle_avoidance():
      """
      This function is used to detect an obstacle, not present while doing the path planning, and overcome it without colliding.
      The Thymio will continue to turn until he reaches the first target after the obstacle, from his global path planning.
    :param obst: a row vector of 5 values from the proximity sensors in front of the Thymio 
    :param position: actual postion of the robot 
    :param path: matrix composed by coordinates (x,y) of the global path planning
    :param saved_pos: index of 
    """
    global prox_horizontal, motor_left_target, motor_right_target, state, obst, obstSpeedGain, speed0 

    # acquisition from the proximity sensors to detect obstacles
    obst = [prox_horizontal[0], prox_horizontal[1], prox_horizontal[2], prox_horizontal[3], prox_horizontal[4]]
    
    position = vision.getRobotPos() # get actual position of the robot
    
    path = PathPlanning.reconstruct_path() # get the next target from Path Planning
    
    saved_pos = 
    
    spLeft = speed0
    spRight = speed0
    # stores the position from path before a local obstacle is detected
    saved_pos = checkState(position, obstThrL, obstThrH)
    saved_pos_idx = 0
    
    for i in range(length(path)):
        if path[i] == saved_pos:
            saved_pos_idx = i
            
    if  state == 0 :
        # goal tracking: turn toward the next target
        motor_left_target = globalNavigation.getMotorLeftSpeed  
        motor_right_target = globalNavigation.getMotorRightSpeed 
    else:
       # adjustment for obstacles
        turn = direction()
        for i in range(5):
            spLeft += prox_horizontal[i] * obstSpeedGain[i] // 100
            spRight += prox_horizontal[i] * obstSpeedGain[4 - i] // 100
 
        if turn[0] == 1:
            if obst[3] < obstThrL:
                if obst[4] < obstThrL:
                    spLeft = 200
                    spRight = 100
                    if position == [path[0, saved_pos_idx], path[1, saved_pos_idx]]:
                        turn[0] = 0
                    if position == [path[0, saved_pos_idx+1], path[1, saved_pos_idx+1]]:
                        turn[0] = 0
                    if position == [path[0, saved_pos_idx+2], path[1, saved_pos_idx+2]]:
                        turn[0] = 0
        if turn[1] == 1:
            if obst[1] < obstThrL:
                if obst[0] < obstThrL:
                    spLeft = 100
                    spRight = 200
                    if position == [path[0, saved_pos_idx], path[1, saved_pos_idx]]:
                        turn[1] = 0
                    if position == [path[0, saved_pos_idx+1], path[1, saved_pos_idx+1]]:
                        turn[1] = 0
                    if position == [path[0, saved_pos_idx+2], path[1, saved_pos_idx+2]:
                        turn[1] = 0
        # motor control
        motor_left_target = spLeft
        motor_right_target = spRight
        

In [ ]:
# Function to evaluate if passing the obstacle clockwise or counterclockwise
def direction():
    """
    Returns the direction to take to overcome the obstacle (clockwise or counterclockwise) 
    :param obst: a row vector of 5 values from the proximity sensors in front of the Thymio 
    :return turn: 1x2 vector containing the clockwise movement in the first column and the counterclockwise movement 
                  in the second column. It stores a binary value 0 if the corresponding movement is locked and 1 if unlocked
    """
    
    # obst = [prox_horizontal[0], prox_horizontal[1], prox_horizontal[2], prox_horizontal[3], prox_horizontal[4]]
    turn = [0, 0]
    # If the obstacle detected is on the right side of the Thymio, then take the clockwise direction,
    # otherwise take the counterclockwise direction
    if (obst[3] + obst[4]) > (obst[0] + obst[1]):
        turn[0] = 1
    elif (obst[3] + obst[4]) < (obst[0] + obst[1]):
        turn[1] = 1
        
    return turn

In [ ]:
def checkState (position, obstThrL, obstThrH)
    """
    Returns the state of the Thymio which indicate whether to follow the path planning 
    from the A* algorithm or to avoid a local obstacle 
    :param obstThrH: the value which the 5 proximity sensors exceed when they detect an object (14 cm)
    :param obstThrL: the value which indicate if the 5 proximity sensors don't detect anything 
    :return saved_pos: position of the Thymio before finding an obstacle 
    """
    global state, obst
    
    if state == 0: 
        # switch from goal tracking to obst avoidance if obstacle detected
        if (obst[0] > obstThrH):
            state = 1
            return saved_pos = position
        elif (obst[1] > obstThrH):
            state = 1
            return saved_pos = position
        elif (obst[2] > obstThrH):
            state = 1
            return saved_pos = position
        elif (obst[3] > obstThrH):
            state = 1
            return saved_pos = position
        elif (obst[4] > obstThrH):
            state = 1  
            return saved_pos = position
            
    elif state == 1:
        # switch from obst avoidance to goal tracking if obstacle got unseen
        if turn[0] == 0: 
            if turn[1] == 0: 
                if obst[0] < obstThrL:
                    if obst[1] < obstThrL:
                        if obst[2] < obstThrL:
                            if obst[3] < obstThrL:
                                if obst[4] < obstThrL:
                                    state = 0
                                    return saved_pos = 0
    